<a href="https://colab.research.google.com/github/evaie/evaie/blob/main/process_inscriptions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get authentication for reading the google sheet

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Read the google sheet and transform it in a dataframe properly


In [ ]:
worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SCruyMO_mncyvIF8kA9HEN2cTabM8ge3Mj2Ia_yG8Uk/edit#gid=464665913').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df_inscriptions = pd.DataFrame.from_records(rows,columns=rows[0])
df_inscriptions = df_inscriptions.drop(0).reindex()

In [ ]:
df_inscriptions['naissance'].count

<bound method Series.count of 1     1958-05-28
2     1984-02-09
3     1962-01-31
4     2008-06-04
5     1965-10-18
         ...    
76    2002-01-25
77    2003-04-24
78    1971-09-09
79    1972-10-22
80              
Name: naissance, Length: 80, dtype: object>

detect the malformed naissance columsns and change the date formate

In [ ]:
df_inscriptions.dropna()
df_inscriptions['naissance'].count()
df_ddl_1 = df_inscriptions[df_inscriptions['naissance']=='0000-00-00']
df_ddl_1

,saison,statut,date_statut,civil,nom,prenom,Licence,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total
23,2021-2022,Validé,2021-08-31 11:49:45,Monsieur,EXBRAYAT,JEAN-LOUIS,oui,jexbrayat@icloud.com,06 69 05 22 63?,71 rue Victor Hugo,92400,Courbevoie,FR,FR,0000-00-00,retrairé,,,,,,,,,,,,Madame,EXBRAYAT,ELYANE,elexbrayat@gmail.com,06 64 26 24 16?,Fanny Exbrayat,2020-09-04,non,on,on,on,on,on,renouvellement,432020,Séniors + Loisirs + CE,non,non,460.00,0.00,,460.00,0.00,0.00,460.00,1,0.00,0.00,0.00,,0.00,0.00,,0.00,460.00


In [ ]:
df_ddl_2 =df_inscriptions[df_inscriptions['naissance']=='']
df_ddl_2

,saison,statut,date_statut,civil,nom,prenom,Licence,email,tel,adresse,cp,ville,pays,nationalite,naissance,profession,entreprise,parent1_civil,parent1_nom,parent1_prenom,parent1_email,parent1_tel,parent2_civil,parent2_nom,parent2_prenom,parent2_email,parent2_tel,contact_civil,contact_nom,contact_prenom,contact_email,contact_tel,docteur_nom,docteur_date,handicap,reglement,documentation,benevolat,nager,assurance,type,licence,categorie,garage,optioncomp,montant,reduction,motifreduction,netapayer,paiement_helloasso,virement,cheque,nbcheques,chequesvacances,couponsport,pass92,caution_nom,caution,especes,autre_details,autre,montant_total
80,,,,,,,34,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='0000-00-00']
df_inscriptions['naissance'].count()

79

In [ ]:
df_inscriptions = df_inscriptions[df_inscriptions['naissance']!='']
df_inscriptions['naissance'].count()

78

In [ ]:
from datetime import datetime
df_inscriptions['naissance']= df_inscriptions[df_inscriptions['naissance']!='0000-00-00']['naissance'].\
apply(lambda x : datetime.strptime(x, '%Y-%m-%d').strftime('%d/%m/%Y'))

In [ ]:
df_inscriptions['naissance']

1     28/05/1958
2     09/02/1984
3     31/01/1962
4     04/06/2008
5     18/10/1965
         ...    
75    30/10/1999
76    25/01/2002
77    24/04/2003
78    09/09/1971
79    22/10/1972
Name: naissance, Length: 78, dtype: object

In [ ]:
try :
  inscription_reprocessed = worksheet.spreadsheet.worksheet('re-processed')
  inscription_reprocessed.clear()
except:
  inscription_reprocessed = worksheet.spreadsheet.add_worksheet(title="re-processed", rows="350", cols="20")

In [ ]:
from gspread_dataframe import set_with_dataframe
set_with_dataframe(inscription_reprocessed, df_inscriptions)